# Running MD simulations using nff and ASE

This Jupyter Notebook shows how the `nff` package interfaces with the Atomistic Simulation Environment (ASE). We assume the user went through tutorial `01_training`, so we can load the pretrained models without having to train them again.

As before, importing the dependencies:

In [7]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from ase import Atoms
from ase.md.verlet import VelocityVerlet

from nff.io import NeuralFF, AtomsBatch
from nff.md.nve import Dynamics
from nff.data import Dataset
from nff.train import load_model, evaluate
import nff.utils.constants as const

## Loading the relevant data

We reload the dataset and create a `GraphLoader` as we did last time:

In [3]:
dataset = Dataset.from_file('data/dataset.pth.tar')

### Creating Atoms

As before, we can create an `Atoms` object from any element of the dataset. Let's take the first one, for simplicity:

In [4]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], numbers=props['nxyz'][:, 0], props=props)

### Creating the ASE calculator

Now we just have to load the ASE calculator from a pretrained model. One way of doing so is through the in-build `from_file` method. You just have to specify the folder where the model was trained and subsequently stored.

In [5]:
nff_ase = NeuralFF.from_file('sandbox/', device=3)

Assigning this calculator to `atoms` is easy:

In [6]:
atoms.set_calculator(nff_ase)

### Configuring the dynamics for the system

In this example, we will run an NVE dynamics simulation. We will use the default parameters there implemented to run a trajectory for an ethanol molecule. The parameters we will specify are the following:

* `T_init`: initial temperature of the simulation
* `time_step`: time step in femtoseconds
* `thermostat`: ASE integrator to use when performing the simulation
* `steps`: number of steps to simulate
* `save_frequency`: how often (in steps) save the pose of the molecule in a file
* `nbr_list_update_freq`: how often (in steps) to update the neighbor list (not yet implemented)
* `thermo_filename`: output file for the thermodynamics log
* `traj_filename`: output file for the ASE trajectory file
* `skip`: number of initial frames to skip when recording the trajectory

In [9]:
md_params = {
    'T_init': 450,
    'time_step': 0.5,
    'thermostat': VelocityVerlet,
    'steps': 200,
    'save_frequency': 10,
    'nbr_list_update_freq': 20,
    'thermo_filename': 'thermo.log',
    'traj_filename': 'atoms.traj',
    'skip': 0
}

In [11]:
nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.4425      -0.1725       0.6150   528.6

0.0050           0.4435      -0.0662       0.5097   438.1

0.0100           0.4443      -0.0663       0.5107   439.0

0.0150           0.4458       0.0551       0.3907   335.9

0.0200           0.4439      -0.1963       0.6401   550.3

0.0250           0.4430      -0.0675       0.5105   438.8

0.0300           0.4435       0.0890       0.3545   304.7

0.0350           0.4430      -0.1357       0.5787   497.4

0.0400           0.4422      -0.1452       0.5874   504.9

0.0450           0.4411      -0.2474       0.6885   591.9

0.0500           0.4409      -0.1460       0.5869   504.5

0.0550           0.4405      -0.1844       0.6249   537.2

0.0600           0.4426      -0.1467       0.5893   506.6

0.0650           0.4441      -0.0564       0.5005   430.2

0.0700           0.4434      -0.0121       0.4554   391.5

0.0750           0.4431      -0.0066       0.4497   386.6


The dynamics conserved the energy. The temperature varied throughout the simulation, as expected.

## Visualizing the trajectory

To visualize the trajectory in this Jupyter Notebook, you will have to install the package [nglview](https://github.com/arose/nglview).

In [12]:
import nglview as nv
from ase.io import Trajectory

_ColormakerRegistry()

Displaying the trajectory:

In [13]:
%matplotlib notebook

traj = Trajectory('atoms.traj')
nv.show_asetraj(traj)

NGLWidget(max_frame=20)

Looks like the atoms are still together. Visual inspection says that the trajectory is reasonable. Yay for `nff`!